In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
#import action chains
from selenium.webdriver.common.action_chains import ActionChains
import time
#import no such element exception
from selenium.common.exceptions import NoSuchElementException
import requests
from lxml import html
import pandas as pd
from datetime import date
import re

In [2]:
#set default options to driver
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1024,768")
# chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--disable-dev-shm-usage")

In [20]:
#initiate selenium driver
driver = webdriver.Chrome(options=chrome_options, executable_path='C:\\chromedriver.exe')

C:\Users\juan-\AppData\Local\Temp\ipykernel_10684\3917679021.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options, executable_path='C:\\chromedriver.exe')


In [16]:
def get_page(url):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    return tree

In [53]:
def get_by_xpath_and_clean(tree, xpath, i=0):
    """"""
    try:
        if i=='join':
            text = tree.xpath(xpath)
            text = re.sub(r'[\n\t\xa0â\x80\x93]', '', ' '.join(text)).strip()
            return text
        else:
            text = tree.xpath(xpath)[i]
            text = re.sub(r'[\n\t\xa0â\x80\x93]', '', text).strip()
            return text
    except:
        return None

In [5]:
words_to_look = [
        'salud',
        'farmacoeconomía',
        'medicamentos',
        'health',
        'pharmacoeconomics',
        'medicines',
        'saude',
        'farmacoeconomia',
        'medicamentos',
    ]

today = date.today().strftime("%d/%m/%Y")
source = 'unops'
file_name = './unops_ops.csv'
main_url = 'https://jobs.unops.org/Pages/ViewVacancy/VAListing.aspx'

In [15]:
#lee la tabla de la pagina
df = pd.read_csv(file_name, encoding='utf-8')

In [28]:
driver.get(main_url)

In [29]:
footer = driver.find_element(By.XPATH, '//div[@class="footer-nav"]')

In [30]:
ActionChains(driver).scroll_to_element(footer).perform()

In [36]:
links = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[contains(@href,"id")]')]

In [37]:
while True:
    try:
        print("-----next page-----")
        driver.find_element(By.XPATH, '(//tr)[last()]/td/span/parent::td/following-sibling::td/a').click()

        print("durmiendo 5 segs")
        time.sleep(5)

        links += [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[contains(@href,"id")]')]
            
    except NoSuchElementException:
        print("no hay mas paginas")
        break

-----next page-----
durmiendo 5 segs
-----next page-----
durmiendo 5 segs
-----next page-----
durmiendo 5 segs
-----next page-----
durmiendo 5 segs
-----next page-----
durmiendo 5 segs
-----next page-----
no hay mas paginas


In [38]:
driver.quit()

In [42]:
detail_page = links[0]
detail_page

'https://jobs.unops.org/Pages/ViewVacancy/VADetails.aspx?id=25673'

In [43]:
detail = get_page(detail_page)

In [63]:
#title
get_by_xpath_and_clean(detail, '//h1/text()')

'Project Management Support  Senior Assistant (two regular positions), Multiple positions'

In [55]:
#location
get_by_xpath_and_clean(detail, '//span[text()="Duty station"]/following-sibling::span/text()')

'Multiple'

In [57]:
#open date
get_by_xpath_and_clean(detail, '//span[text()="Application period"]/following-sibling::span/text()').split("to")[0].strip()

'17-Mar-2023'

In [58]:
#closing date
get_by_xpath_and_clean(detail, '//span[text()="Application period"]/following-sibling::span/text()').split("to")[1].strip()

'09-Apr-2023'

In [62]:
#text for search
get_by_xpath_and_clean(detail, '(//span[@class="longTextDescription"])[1]//descendant::text()', i='join')

'The PRO-Local Governance for People and Nature Programme, is designed to contribute to improving the well-being and quality of life of citizens through improved local governance, social inclusion and environmental protection in 99 cities and municipalities in two regions: Šumadija and Western Serbia and Southern and Eastern Serbia. The Government of Switzerland has allocated CHF 9 million to fund the Programme which will be implemented by the United Nations Office for Project Services (UNOPS) in partnership with UN agencies (1) over a period of 48 months.  To achieve the Programme’s Objective, the intervention will focus on three key Pillars and Outcomes:            Pillar Good Governance, Outcome 1: LGs improve capacities and apply good governance principles in practice;                Pillar Social Inclusion, Outcome 2: LGs and other relevant local actors improve capacities and develop evidence-based local social protection policies;                Pillar Environmental Governance, O

In [66]:
for i, link in enumerate(links):
    #get the url of detail
    detail_url = link
    
    #looks if the detail url is already in the dataset
    if df['url_detail_id'][df['url_detail_id']==detail_url].any():
        print('this job is already in the dataset')
        continue

    #if not  exist, get the detail
    else:
        print('nueva oportunidad encontrada')

        detail_page = get_page(detail_url)
        
        #get the title
        title = get_by_xpath_and_clean(detail_page, '//h1/text()')
        
        #get the location (type RFQ)
        location = get_by_xpath_and_clean(detail_page, '//span[text()="Duty station"]/following-sibling::span/text()')
        #Opening Date:
        opening_date = get_by_xpath_and_clean(detail_page, 
            '//span[text()="Application period"]/following-sibling::span/text()').split("to")[0].strip()
        #Closing Date
        closing_date = get_by_xpath_and_clean(detail_page, 
            '//span[text()="Application period"]/following-sibling::span/text()').split("to")[1].strip()
        
        #find the body of the job and look for the words_to_look to appear once at least
        is_alert = False

        #text for search
        text_for_alert = title + get_by_xpath_and_clean(detail_page, 
            '(//span[@class="longTextDescription"])[1]//descendant::text()', i='join')
        if any(word in text_for_alert for word in words_to_look):
            is_alert = True
            
        #add the new job to the dataset
        df = df.append({'url_detail_id': detail_url, 'scrapped_day': today,  'title': title, 
                'opening_date': opening_date, 'closing_date': closing_date,'location': location,
                'is_alert': is_alert, 'source': source}, ignore_index=True)


this job is already in the dataset
nueva oportunidad encontrada
nueva oportunidad encontrada
nueva oportunidad encontrada
nueva oportunidad encontrada


KeyboardInterrupt: 

In [65]:
df

,url_detail_id,scrapped_day,title,opening_date,closing_date,source,is_alert,location
0,https://jobs.unops.org/Pages/ViewVacancy/VADet...,19/03/2023,Project Management Support Senior Assistant (...,17-Mar-2023,09-Apr-2023,unops,False,Multiple
